Você pode adquirir versões impressas e de e-book do *Think Python 3e* (em inglês) em
[Bookshop.org](https://bookshop.org/a/98697/9781098155438) e
[Amazon](https://www.amazon.com/_/dp/1098155432?smid=ATVPDKIKX0DER&_encoding=UTF8&tag=oreilly20-20&_encoding=UTF8&tag=greenteapre01-20&linkCode=ur2&linkId=e2a529f94920295d27ec8a06e757dc7c&camp=1789&creative=9325).

Uma versão em língua portuguesa da 3ª edição foi publicada pela editora [Novatec](https://novatec.com.br/livros/pense-em-python-3ed/).

In [ ]:
from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + str(local))
    return filename

download('https://github.com/AllenDowney/ThinkPython/raw/v3/thinkpython.py');
download('https://github.com/AllenDowney/ThinkPython/raw/v3/diagram.py');

import thinkpython

# *Strings* e Expressões Regulares

*Strings* não são como inteiros, pontos flutuantes e booleanos. Uma *string* é uma **sequência**, o que significa que ela contém vários valores em uma ordem específica.
Neste capítulo, veremos como acessar os valores que compõem uma *string* e usaremos funções que processam *strings*.

Também usaremos expressões regulares, que são uma ferramenta poderosa para encontrar padrões em uma *string* e executar operações como pesquisar e substituir.

Como exercício, você terá a chance de aplicar essas ferramentas a um jogo de palavras chamado Wordle.

## Uma *string* é uma sequência

Uma *string* é uma sequência de caracteres. Um **caractere** pode ser uma letra (em quase qualquer alfabeto), um dígito, um sinal de pontuação ou um espaço em branco.

Você pode selecionar um caractere de uma *string* com o operador de colchetes.
Esta instrução de exemplo seleciona o caractere número 1 de `fruit` e
o atribui a `letter`:

In [ ]:
fruit = 'banana'
letter = fruit[1]

A expressão entre colchetes é um **índice**, assim chamado porque *indica* qual caractere na sequência selecionar.
Mas o resultado pode não ser o que você espera:

In [ ]:
letter

A letra com índice `1` é, na verdade, a segunda letra da *string*.
Um índice é um deslocamento do início da *string*, então o deslocamento da primeira letra é `0`:

In [ ]:
fruit[0]

Você pode pensar em `'b'` como a 0ª letra de `'banana'` -- pronunciado "zerézima".

O índice entre colchetes pode ser uma variável:

In [ ]:
i = 1
fruit[i]

Ou uma expressão que contém variáveis ​​e operadores:

In [ ]:
fruit[i+1]

Mas o valor do índice tem que ser um inteiro -- caso contrário, você receberá um `TypeError`:

In [ ]:
%%expect TypeError

fruit[1.5]

Como vimos no Capítulo 1, podemos usar a função interna `len` para obter o comprimento de uma *string*:

In [ ]:
n = len(fruit)
n

Para obter a última letra de uma *string*, você pode ficar tentado a escrever isto:

In [ ]:
%%expect IndexError

fruit[n]

Mas isso causa um `IndexError` porque não há nenhuma letra em `'banana'` com o índice 6. Como começamos a contar em `0`, as seis letras são numeradas de `0` a `5`. Para obter o último caractere, você tem que subtrair `1` de `n`:

In [ ]:
fruit[n-1]

Mas há uma maneira mais fácil.
Para obter a última letra em uma string, você pode usar um índice negativo, que conta para trás a partir do final:

In [ ]:
fruit[-1]

O índice `-1` seleciona a última letra, `-2` seleciona a penúltima, e assim por diante.

## Fatiamento de *strings*

Um segmento de uma *string* é chamado de **fatia**.
Selecionar uma fatia é semelhante a selecionar um caractere:

In [ ]:
fruit = 'banana'
fruit[0:3]

O operador `[n:m]` devolve a parte da *string* do `n`ésimo
caractere ao `m`ésimo caractere, incluindo o primeiro, mas excluindo o segundo.
Esse comportamento é contraintuitivo, mas pode ajudar imaginar os índices apontando *entre* os caracteres, como nesta figura:

In [ ]:
from diagram import make_binding, Element, Value

binding = make_binding("fruit", ' b a n a n a ')
elements = [Element(Value(i), None) for i in range(7)]

In [ ]:
import matplotlib.pyplot as plt
from diagram import diagram, adjust
from matplotlib.transforms import Bbox

width, height, x, y = [1.35, 0.54, 0.23, 0.39]

ax = diagram(width, height)
bbox = binding.draw(ax, x, y)
bboxes = [bbox]

def draw_elts(x, y, elements):
    for elt in elements:
        bbox = elt.draw(ax, x, y, draw_value=False)
        bboxes.append(bbox)

        x1 = (bbox.xmin + bbox.xmax) / 2
        y1 = bbox.ymax + 0.02
        y2 = y1 + 0.14
        handle = plt.plot([x1, x1], [y1, y2], ':', lw=0.5, color='gray')
        x += 0.105

draw_elts(x + 0.48, y - 0.25, elements)
bbox = Bbox.union(bboxes)
# adjust(x, y, bbox)

Por exemplo, a fatia `[3:6]` seleciona as letras `ana`, o que significa que `6` é legal como parte de uma fatia, mas não é legal como um índice.

Se você omitir o primeiro índice, a fatia começa no início da *string*:

In [ ]:
fruit[:3]

Se você omitir o segundo índice, a fatia vai até o final da *string*:

In [ ]:
fruit[3:]

Se o primeiro índice for maior ou igual ao segundo, o resultado será uma ***string* vazia**, representada por duas aspas:

In [ ]:
fruit[3:3]

Uma *string* vazia não contém caracteres e tem comprimento 0.

Continuando este exemplo, o que você acha que `fruit[:]` significa? Tente e veja:

In [ ]:
fruit[:]

## *Strings* são imutáveis

É tentador usar o operador `[]` no lado esquerdo de uma
atribuição, com a intenção de alterar um caractere em uma string, assim:

In [ ]:
%%expect TypeError

greeting = 'Hello, world!'
greeting[0] = 'J'

O resultado é um `TypeError`.
Na mensagem de erro, o "objeto" é a *string* e o "item" é o caractere
que tentamos atribuir.
Por enquanto, um **objeto** é a mesma coisa que um valor, mas refinaremos essa definição mais tarde.

O motivo desse erro é que as *strings* são **imutáveis**, o que significa que você não pode alterar uma *string* existente.
O melhor que você pode fazer é criar uma nova *string* que seja uma variação da original:

In [ ]:
new_greeting = 'J' + greeting[1:]
new_greeting

Este exemplo concatena uma nova primeira letra em uma fatia de `greeting`.
Não tem efeito na *string* original:

In [ ]:
greeting

## Comparação de *strings*

Os operadores relacionais funcionam em *strings*. Para ver se duas *strings* são iguais, podemos usar o operador `==`:

In [ ]:
word = 'banana'

if word == 'banana':
    print('All right, banana.')

Outras operações relacionais são úteis para colocar palavras em ordem
alfabética:

In [ ]:
def compare_word(word):
    if word < 'banana':
        print(word, 'comes before banana.')
    elif word > 'banana':
        print(word, 'comes after banana.')
    else:
        print('All right, banana.')

In [ ]:
compare_word('apple')

Python não lida com letras maiúsculas e minúsculas da mesma forma que as pessoas. Todas as letras maiúsculas vêm antes de todas as letras minúsculas, então:

In [ ]:
compare_word('Pineapple')

Para resolver esse problema, podemos converter strings para um formato padrão, como todas minúsculas, antes de executar a comparação.
Tenha isso em mente se você tiver que se defender de um homem armado com um Pineapple.

## Métodos de *string*

*Strings* fornecem métodos que realizam uma variedade de operações úteis.
Um método é semelhante a uma função -- ele recebe argumentos e devolve um valor -- mas a sintaxe é diferente.
Por exemplo, o método `upper` recebe uma *string* e devolve uma nova *string* com todas as letras maiúsculas.

Em vez da sintaxe de função `upper(word)`, ele usa a sintaxe de método `word.upper()`:

In [ ]:
word = 'banana'
new_word = word.upper()
new_word

Este uso do operador ponto especifica o nome do método, `upper`, e o nome da *string* para aplicar o método, `word`.
Os parênteses vazios indicam que este método não recebe argumentos.

Uma chamada de método é chamada de **invocação**; neste caso, diríamos que estamos invocando `upper` em `word`.

## Escrevendo arquivos

Operadores e métodos de *strings* são úteis para ler e escrever arquivos de texto. Como exemplo, trabalharemos com o texto de *Drácula*, um romance de Bram Stoker que está disponível no Projeto Gutenberg (<https://www.gutenberg.org/ebooks/345>) (em inglês):

In [ ]:
import os

if not os.path.exists('pg345.txt'):
    !wget https://www.gutenberg.org/cache/epub/345/pg345.txt

Baixei o livro em um arquivo de texto simples chamado `pg345.txt`, que podemos abrir para leitura assim:

In [ ]:
reader = open('pg345.txt')

Além do texto do livro, este arquivo contém uma seção no início com informações sobre o livro e uma seção no final com informações sobre a licença.
Antes de processarmos o texto, podemos remover esse material extra encontrando as linhas especiais no início e no fim que começam com `'***'`.

A função a seguir pega uma linha e verifica se é uma das linhas especiais.
Ele usa o método `startswith`, que verifica se uma *string* começa com uma determinada sequência de caracteres.

In [ ]:
def is_special_line(line):
    return line.startswith('*** ')

Podemos usar esta função para percorrer as linhas do arquivo e exibir apenas as linhas especiais:

In [ ]:
for line in reader:
    if is_special_line(line):
        print(line.strip())

Agora vamos criar um novo arquivo, chamado `pg345_cleaned.txt`, que contém apenas o texto do livro.
Para percorrer o livro novamente, temos que abrí-lo novamente para leitura. E, para escrever um novo arquivo, podemos abrí-lo para escrita:

In [ ]:
reader = open('pg345.txt')
writer = open('pg345_cleaned.txt', 'w')

`open` recebe um parâmetro opcional que especifica o "modo" -- neste exemplo, `'w'` indica que estamos abrindo o arquivo para escrita.
Se o arquivo não existir, ele será criado; se já existir, o conteúdo será substituído.

Como primeiro passo, percorreremos o arquivo até encontrarmos a primeira linha especial:

In [ ]:
for line in reader:
    if is_special_line(line):
        break

A instrução `break` "pula fora" do laço -- isto é, faz com que o laço termine imediatamente, antes de chegarmos ao fim do arquivo.

Quando o laço sai, `line` contém a linha especial que tornou a condicional verdadeira:

In [ ]:
line

Como `reader` mantém o controle de onde está no arquivo, podemos usar um segundo laço para continuar de onde paramos.

O laço a seguir lê o restante do arquivo, uma linha de cada vez.
Quando ele encontra a linha especial que indica o fim do texto, ele sai do laço.
Caso contrário, ele grava a linha no arquivo de saída:

In [ ]:
for line in reader:
    if is_special_line(line):
        break
    writer.write(line)

Quando esse laço termina, `line` contém a segunda linha especial.

In [ ]:
line

Neste ponto, `reader` e `writer` ainda estão abertos, o que significa que poderíamos continuar lendo linhas de `reader` ou escrevendo linhas para `writer`.
Para indicar que terminamos, podemos fechar ambos os arquivos invocando o método `close`:

In [ ]:
reader.close()
writer.close()

Para verificar se esse processo foi bem-sucedido, podemos ler as primeiras linhas do novo arquivo que acabamos de criar:

In [ ]:
for line in open('pg345_cleaned.txt'):
    line = line.strip()
    if len(line) > 0:
        print(line)
    if line.endswith('Stoker'):
        break

O método `endswith` verifica se uma *string* termina com uma determinada sequência de caracteres.

## Localizar e substituir

Na tradução islandesa de *Drácula* de 1901, o nome de um dos personagens foi alterado de "Jonathan" para "Thomas".
Para fazer essa alteração na versão em inglês, podemos percorrer o livro, usar o método `replace` para substituir um nome por outro e gravar o resultado em um novo arquivo.

Começaremos contando as linhas na versão limpa do arquivo:

In [ ]:
total = 0
for line in open('pg345_cleaned.txt'):
    total += 1

total

Para ver se uma linha contém "Jonathan", podemos usar o operador `in`, que verifica se essa sequência de caracteres aparece em algum lugar da linha:

In [ ]:
total = 0
for line in open('pg345_cleaned.txt'):
    if 'Jonathan' in line:
        total += 1

total

Há 199 linhas que contêm o nome, mas esse não é o número total de vezes que o nome aparece, porque ele pode aparecer mais de uma vez em uma linha.
Para obter o total, podemos usar o método `count`, que retorna o número de vezes que uma sequência aparece em uma *string*:

In [ ]:
total = 0
for line in open('pg345_cleaned.txt'):
    total += line.count('Jonathan')

total

Agora podemos substituir `'Jonathan'` por `'Thomas'` assim:

In [ ]:
writer = open('pg345_replaced.txt', 'w')

for line in open('pg345_cleaned.txt'):
    line = line.replace('Jonathan', 'Thomas')
    writer.write(line)

O resultado é um novo arquivo chamado `pg345_replaced.txt` que contém uma versão de *Drácula* onde Jonathan Harker é chamado de Thomas:

In [ ]:
total = 0
for line in open('pg345_replaced.txt'):
    total += line.count('Thomas')

total

## Expressões regulares

Se soubermos exatamente qual sequência de caracteres estamos procurando, podemos usar o operador `in` para encontrá-la e o método `replace` para substituí-la.
Mas há outra ferramenta, chamada de **expressão regular**, que também pode executar essas operações -- e muito mais.

Para demonstrar, começarei com um exemplo simples e trabalharemos em direção a exemplos mais complexos.
Suponha, novamente, que queremos encontrar todas as linhas que contêm uma palavra específica.
Para variar, vamos procurar referências ao personagem titular do livro, o Conde Drácula.
Aqui está uma linha que o menciona:

In [ ]:
text = "I am Dracula; and I bid you welcome, Mr. Harker, to my house."

E aqui está o **padrão** que usaremos para pesquisar:

In [ ]:
pattern = 'Dracula'

Um módulo chamado `re` fornece funções relacionadas a expressões regulares.
Podemos importá-lo assim e usar a função `search` para verificar se o padrão aparece no texto:

In [ ]:
import re

result = re.search(pattern, text)
result

Se o padrão aparecer no texto, `search` devolve um objeto `Match` que contém os resultados da busca.
Entre outras informações, ele tem uma variável chamada `string` que contém o texto que foi pesquisado:

In [ ]:
result.string

Ele também fornece um método chamado `group` que devolve a parte do texto que corresponde ao padrão.

In [ ]:
result.group()

E fornece um método chamado `span` que devolve o índice no texto onde o padrão começa e termina:

In [ ]:
result.span()

Se o padrão não aparecer no texto, o valor de devolvido por `search` será `None`:

In [ ]:
result = re.search('Count', text)
print(result)

Assim, podemos verificar se a busca foi bem-sucedida verificando se o resultado é `None`.

In [ ]:
result == None

Juntando tudo isso, aqui está uma função que percorre as linhas do livro até encontrar uma que corresponda ao padrão fornecido e devolve o objeto `Match`:

In [ ]:
def find_first(pattern):
    for line in open('pg345_cleaned.txt'):
        result = re.search(pattern, line)
        if result != None:
            return result

Podemos usá-la para encontrar a primeira menção de um personagem:

In [ ]:
result = find_first('Harker')
result.string

Para este exemplo, não precisávamos usar expressões regulares -- poderíamos ter feito a mesma coisa mais facilmente com o operador `in`.
Mas expressões regulares podem fazer coisas que o operador `in` não pode.

Por exemplo, se o padrão incluir o caractere de barra vertical, `'|'`, ele pode corresponder à sequência à esquerda ou à sequência à direita.
Suponha que queremos encontrar a primeira menção de Mina Murray no livro, mas não temos certeza se ela é referida pelo primeiro nome ou sobrenome.
Podemos usar o padrão a seguir, que corresponde a qualquer nome:

In [ ]:
pattern = 'Mina|Murray'
result = find_first(pattern)
result.string

Podemos usar um padrão como esse para ver quantas vezes um personagem é mencionado por qualquer nome.
Aqui está uma função que percorre o livro e conta o número de linhas que correspondem ao padrão fornecido:

In [ ]:
def count_matches(pattern):
    count = 0
    for line in open('pg345_cleaned.txt'):
        result = re.search(pattern, line)
        if result != None:
            count += 1
    return count

Agora vamos ver quantas vezes Mina é mencionada:

In [ ]:
count_matches('Mina|Murray')

O caractere especial `'^'` corresponde ao início de uma *string*, assim podemos encontrar uma linha que começa com um determinado padrão:

In [ ]:
result = find_first('^Dracula')
result.string

E o caractere especial `'$'` corresponde ao final de uma string, assim podemos encontrar uma linha que termina com um determinado padrão (ignorando a nova linha no final).

In [ ]:
result = find_first('Harker$')
result.string

## Substituição de *strings*

Bram Stoker nasceu na Irlanda, e quando *Drácula* foi publicado em 1897, ele estava morando na Inglaterra.
Então, esperaríamos que ele usasse a grafia britânica de palavras como "centre" e "colour".
Para verificar, podemos usar o seguinte padrão, que corresponde a "centre" ou à grafia americana "center":

In [ ]:
pattern = 'cent(er|re)'

Neste padrão, os parênteses envolvem a parte do padrão à qual a barra vertical se aplica.
Então, este padrão corresponde a uma sequência que começa com `'cent'` e termina com `'er'` ou `'re'`:

In [ ]:
result = find_first(pattern)
result.string

Como esperado, ele usou a grafia britânica.

Também podemos verificar se ele usou a grafia britânica de "colour".
O padrão a seguir usa o caractere especial `'?'`, o que significa que o caractere anterior é opcional:

In [ ]:
pattern = 'colou?r'

Este padrão combina "colour" com o `'u'` ou "color" sem ele:

In [ ]:
result = find_first(pattern)
line = result.string
line

Novamente, como esperado, ele usou a grafia britânica.

Agora, suponha que queremos produzir uma edição do livro com grafia americana.
Podemos usar a função `sub` no módulo `re`, que faz **substituição de *strings***.

In [ ]:
re.sub(pattern, 'color', line)

O primeiro argumento é o padrão que queremos encontrar e substituir, o segundo é o que queremos substituir e o terceiro é a *string* que queremos pesquisar.
No resultado, você pode ver que "colour" foi substituído por "color":

In [ ]:
# Usei esta função para procurar linhas para usar como exemplos

def all_matches(pattern):
    for line in open('pg345_cleaned.txt'):
        result = re.search(pattern, line)
        if result:
            print(line.strip())

In [ ]:
# Aqui está o padrão que usei (que usa alguns recursos que não vimos)

names = r'(?<!\.\s)[A-Z][a-zA-Z]+'

all_matches(names)

## Depuração

Quando você está lendo e escrevendo arquivos, a depuração pode ser complicada.
Se você estiver trabalhando em um notebook Jupyter, você pode usar **comandos *shell*** para ajudar.
Por exemplo, para exibir as primeiras linhas de um arquivo, você pode usar o comando `!head`, assim:

In [ ]:
!head pg345_cleaned.txt

O ponto de exclamação inicial, `!`, indica que este é um comando *shell*, que não faz parte do Python.
Para exibir as últimas linhas, você pode usar `!tail`:

In [ ]:
!tail pg345_cleaned.txt

Ao trabalhar com arquivos grandes, a depuração pode ser difícil porque pode haver muita saída para verificar manualmente.
Uma boa estratégia de depuração é começar com apenas parte do arquivo, fazer o programa funcionar e, em seguida, executá-lo com o arquivo inteiro.

Para criar um arquivo pequeno que contenha parte de um arquivo maior, podemos usar `!head` novamente com o operador de redirecionamento, `>`, que indica que os resultados devem ser gravados em um arquivo em vez de exibidos:

In [ ]:
!head pg345_cleaned.txt > pg345_cleaned_10_lines.txt

Por padrão, `!head` lê as primeiras 10 linhas, mas recebe um argumento opcional que indica o número de linhas a serem lidas:

In [ ]:
!head -100 pg345_cleaned.txt > pg345_cleaned_100_lines.txt

Este comando *shell* lê as primeiras 100 linhas de `pg345_cleaned.txt` e as grava em um arquivo chamado `pg345_cleaned_100_lines.txt`.

Nota: Os comandos *shell* `!head` e `!tail` não estão disponíveis em todos os sistemas operacionais.
Se eles não funcionarem para você, podemos escrever funções semelhantes em Python.
Veja o primeiro exercício no final deste capítulo para sugestões.

## Glossário

**sequência** (*sequence*)**:**
Uma coleção ordenada de valores onde cada valor é identificado por um índice inteiro.

**caractere** (*character*)**:**
Um elemento de uma *string*, incluindo letras, números e símbolos.

**índice** (*index*)**:**
Um valor inteiro usado para selecionar um item em uma sequência, como um caractere em uma *string*. Em Python, os índices começam em `0`.

**fatia** (*slice*)**:**
Uma parte de uma *string* especificada por um intervalo de índices.

***string* vazia** (*empty string*)**:**
Uma *string* que não contém caracteres e tem comprimento `0`.

**objeto** (*object*)**:**
Algo a que uma variável pode se referir. Um objeto tem um tipo e um valor.

**imutável** (*immutable*)**:**
Se os elementos de um objeto não podem ser alterados, o objeto é imutável.

**invocação** (*invocation*)**:**
Uma expressão -- ou parte de uma expressão -- que chama um método.

**expressão regular** (*regular expression*)**:**
Uma sequência de caracteres que define um padrão de pesquisa.

**padrão** (*pattern*)**:**
Uma regra que especifica os requisitos que uma *string* tem que atender para constituir uma correspondência.

**substituição de string** (*string substitution*)**:**
Substituição de uma *string*, ou parte de uma *string*, por outra *string*.

**comando *shell***(*shell command*)**:**
Uma declaração em uma linguagem *shell*, que é uma linguagem usada para interagir com um sistema operacional.

## Exercícios

In [ ]:
# Esta célula diz ao Jupyter para fornecer informações detalhadas de depuração
# quando ocorre um erro de tempo de execução. Execute-o antes de trabalhar nos
# exercícios.

%xmode Verbose

In [ ]:
download('https://raw.githubusercontent.com/AllenDowney/ThinkPython/v3/words.txt');

### Pergunte a um assistente virtual

Neste capítulo, nós apenas arranhamos a superfície do que as expressões regulares podem fazer.
Para ter uma ideia do que é possível, pergunte a um assistente virtual: "Quais são os caracteres especiais mais comuns usados ​​em expressões regulares Python?" ("*What are the most common special characters used in Python regular expressions?*").

Você também pode pedir um padrão que corresponda a tipos específicos de *strings*.
Por exemplo, tente pedir:

* Escreva uma expressão regular Python que corresponda a um número de telefone de 10 dígitos com hifens. (*Write a Python regular expression that matches a 10-digit phone number with hyphens.*)

* Escreva uma expressão regular Python que corresponda a um endereço de rua com um número e um nome de rua, seguido por `RUA` ou `AV.`. (*Write a Python regular expression that matches a street address with a number and a street name, followed by `ST` or `AVE`.*)

NOTA do TRADUTOR: no exercício anterior, embora não seja o padrão no Brasil, foi mantida a orientação de colocar RUA e AV. no final em função do objetivo do exercício.

* Escreva uma expressão regular Python que corresponda a um nome completo com qualquer título comum como `Sr.` ou `Sr.a` seguido por qualquer número de nomes começando com letras maiúsculas, possivelmente com hifens entre alguns nomes. (* Write a Python regular expression that matches a full name with any common title like `Mr` or `Mrs` followed by any number of names beginning with capital letters, possibly with hyphens between some names.*)

E se você quiser ver algo mais complicado, tente pedir uma expressão regular que corresponda a qualquer URL legal.

Uma expressão regular geralmente tem a letra `r` antes das aspas, o que indica que é uma "string bruta".
Para mais informações, pergunte a um assistente virtual: "O que é uma string bruta em Python?" ("*What is a raw string in Python?*").

In [ ]:
from doctest import run_docstring_examples

def run_doctests(func):
    run_docstring_examples(func, globals(), name=func.__name__)

### Exercício

Veja se você consegue escrever uma função que faça a mesma coisa que o comando *shell* `!head`.
Ela deve receber como argumentos o nome de um arquivo para ler, o número de linhas para ler e o nome do arquivo onde gravar as linhas.
Se o terceiro parâmetro for `None`, ela deve exibir as linhas em vez de gravá-las em um arquivo.

Considere pedir ajuda a um assistente virtual, mas se fizer isso, diga a ele para não usar uma instrução `with` ou uma instrução `try`.

In [ ]:
# A solução vai aqui

Você pode usar os exemplos a seguir para testar sua função:

In [ ]:
head('pg345_cleaned.txt', 10)

In [ ]:
head('pg345_cleaned.txt', 100, 'pg345_cleaned_100_lines.txt')

In [ ]:
!tail pg345_cleaned_100_lines.txt

### Exercício

"Wordle" é um jogo de palavras online em que o objetivo é adivinhar uma palavra de cinco letras em seis ou menos tentativas.
Cada tentativa deve ser reconhecida como uma palavra, sem incluir nomes próprios.
Após cada tentativa, você obtém informações sobre quais das letras que você adivinhou aparecem na palavra-alvo e quais estão na posição correta.

Por exemplo, suponha que a palavra-alvo seja `MOWER` e você adivinhe `TRIED`.
Você aprenderia que `E` está na palavra e na posição correta, `R` está na palavra, mas não na posição correta, e `T`, `I` e `D` não estão na palavra.

Como um exemplo diferente, suponha que você adivinhou as palavras `SPADE` e `CLERK`, e aprendeu que `E` está na palavra, mas não em nenhuma dessas posições, e nenhuma das outras letras aparece na palavra.
Das palavras na lista de palavras, quantas poderiam ser a palavra-alvo?
Escreva uma função chamada `check_word` que pega uma palavra de cinco letras e verifica se ela pode ser a palavra alvo, dados esses palpites.

In [ ]:
# A solução vai aqui

Você pode usar qualquer uma das funções do capítulo anterior, como `uses_any`:

In [ ]:
def uses_any(word, letters):
    for letter in word.lower():
        if letter in letters.lower():
            return True
    return False

Você pode usar o seguinte laço de repetição para testar sua função:

In [ ]:
for line in open('words.txt'):
    word = line.strip()
    if len(word) == 5 and check_word(word):
        print(word)

### Exercício

Continuando o exercício anterior, suponha que você adivinhe a palavra `TOTEM` e aprenda que o `E` *ainda* não está no lugar certo, mas o `M` está. Quantas palavras sobraram?

In [ ]:
# A solução vai aqui

In [ ]:
# A solução vai aqui

### Exercício

*O Conde de Monte Cristo* é um romance de Alexandre Dumas que é considerado um clássico.
No entanto, na introdução de uma tradução em inglês do livro, o escritor Umberto Eco confessa que achou o livro "um dos romances mais mal escritos de todos os tempos".

Em particular, ele diz que é "desavergonhado em sua repetição do mesmo adjetivo" e menciona em particular o número de vezes que "seus personagens estremecem (*shudders*) ou empalidecem (*turn pale*)".

Para ver se sua objeção é válida, vamos contar o número de linhas que contêm a palavra `pale` em qualquer forma, incluindo `pale`, `pales`, `paled` e `paleness`, bem como a palavra relacionada `pallor`.
Use uma única expressão regular que corresponda a qualquer uma dessas palavras.
Como um desafio adicional, certifique-se de que ela não corresponda a nenhuma outra palavra, como `impale` -- você pode pedir ajuda a um assistente virtual.

A célula a seguir baixa o livro do Projeto Gutenberg <https://www.gutenberg.org/ebooks/1184> (em inglês).

In [ ]:
import os

if not os.path.exists('pg1184.txt'):
    !wget https://www.gutenberg.org/cache/epub/1184/pg1184.txt

A célula a seguir executa uma função que lê o arquivo do Projeto Gutenberg e grava um arquivo que contém apenas o texto do livro, não as informações adicionadas sobre o livro.

In [ ]:
def clean_file(input_file, output_file):
    reader = open(input_file)
    writer = open(output_file, 'w')

    for line in reader:
        if is_special_line(line):
            break

    for line in reader:
        if is_special_line(line):
            break
        writer.write(line)

    reader.close()
    writer.close()

clean_file('pg1184.txt', 'pg1184_cleaned.txt')

In [ ]:
# A solução vai aqui

In [ ]:
# A solução vai aqui

In [ ]:
# A solução vai aqui

Por essa contagem, essas palavras aparecem em `223` linhas do livro, então o Sr. Eco pode ter razão.

[Pense Python: 3ª Edição](https://rodrigocarlson.github.io/PensePython3ed/)

Copyright 2024 [Allen B. Downey](https://allendowney.com/) (versão original)

Copyright 2025 [Rodrigo Castelan Carlson](https://rodrigocarlson.paginas.ufsc.br/) (desta versão)

Foram preservadas as mesmas licenças da versão original.

Licença dos códigos: [MIT License](https://mit-license.org/)

Licença dos textos: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/)